In [2]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (TimeoutException,
                                        ElementNotVisibleException, 
                                        ElementNotSelectableException, 
                                        NoSuchElementException, 
                                        ElementNotInteractableException, 
                                        StaleElementReferenceException, 
                                        ElementClickInterceptedException, 
                                        InvalidSelectorException, 
                                        WebDriverException,
                                        NoSuchWindowException
                                        )
from urllib3.exceptions import NewConnectionError, MaxRetryError
from typing import NamedTuple
from zipfile import ZipFile, ZIP_DEFLATED
import os, signal, sys, psutil
import re
from time import sleep as time_sleep, time
from datetime import datetime
from collections import deque
import pickle
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import logging
from pathlib import Path

In [3]:
# получение пользовательского логгера и установка уровня логирования
short_logger = logging.getLogger('short_logger')
short_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
short_handler = logging.FileHandler("data/short_logger.log", encoding='utf-8', mode='a')
short_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
short_handler.setFormatter(short_formatter)
# добавление обработчика к логгеру
short_logger.addHandler(short_handler)

# получение пользовательского логгера и установка уровня логирования
long_logger = logging.getLogger('long_logger')
long_logger.setLevel(logging.WARNING)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
long_handler = logging.FileHandler("data/long_logger.log", encoding='utf-8', mode='w')
long_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
long_handler.setFormatter(long_formatter)
# добавление обработчика к логгеру
long_logger.addHandler(long_handler)

In [4]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H-%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + " " + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')
    

def occupied_memory() -> float:
    # Получить список всех процессов
    all_processes = psutil.process_iter()

    # Пройтись по каждому процессу и найти процессы с именем "chrome.exe"
    chrome_processes = [p for p in all_processes if p.name() == "chrome.exe"]

    # Инициализировать переменную для хранения общего объема памяти
    total_memory = 0

    # Получить информацию о памяти для каждого процесса Chrome

    if chrome_processes:
        for process in chrome_processes:
            try:
                memory_info = process.memory_info()
            except psutil.NoSuchProcess:
                memory_usage = 0
            else:
                memory_usage = memory_info.rss
                total_memory += memory_usage

    # Преобразовать размер в удобочитаемый формат
    # formatted_size = psutil._common.bytes2human(total_memory)
    return (total_memory) / (1024**2)

def kill_chrome():
    # Получить список всех процессов
    all_processes = psutil.process_iter()

    # Пройтись по каждому процессу и найти процессы с именем "chrome.exe"
    chrome_processes = [p for p in all_processes if p.name() == "chrome.exe"]

    # Получить информацию о памяти для каждого процесса Chrome

    if chrome_processes:
        for process in chrome_processes:
            try:
                pid = process.pid
            except psutil.NoSuchProcess:
                pass
            else:
                os.kill(pid, signal.SIGINT)
                


def reload_driver(driver: webdriver.Chrome, time_wait: int=5) -> webdriver.Chrome:
    if occupied_memory() > 0:
        kill_chrome()
    driver = webdriver.Chrome()
    driver.maximize_window()
    while occupied_memory() <= 1:
        time_sleep(0.5)
    time_sleep(time_wait)
    return driver

def _check_load_page(driver: webdriver.Chrome, timeout: int=20, limitMB: int=1400) -> bool:
    """
       Проверка наличия названия продукта в ячейке
    """
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        n = driver.find_elements(By.XPATH, """//div[contains(@class, "p-dsk-srch-retailer__content")]
                                                //div[contains(@class, "b-dsk-grid__container")]
                                                /a[contains(@class, "p-dsk-srch-retailer__card")]
                                                //div[contains(@class, "b-srch-card__price-new")]
                                                //span[@data-test-ref="money-base"]
                                                """)
        
        # Проверка на переполнение памяти:
        if occupied_memory() > limitMB:
            raise WebDriverMemoryOut('Превышение лимита')
            
        if len(n) > 0:
            if int(n[0].text.replace('\xa0', '').replace(' ', '')) > 0:
                return True
        else:
            time_sleep(0.2)
            continue
    return False

In [5]:
class market_link(NamedTuple):
    sity: str
    market: str
    discounts: int
    data_uuid: str
    href: str


class MyNoData(Exception):
    def __init__(self, text: str='no data'):
        self.txt = text

class MyTimeoutError(Exception):
    def __init__(self, text: str='timeout'):
        self.txt = text

class MyLinkDifferError(Exception):
    def __init__(self, text: str='Link Differ'):
        self.txt = text

class WebDriverMemoryOut(Exception):
    def __init__(self, text: str='Web Driver Memory Out'):
        self.txt = text

In [22]:
def get_pages_links(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    
    def data_parse(driver, wait, url, timeout):
        time_start = int(time())
        while (int(time()) - time_start) < timeout:
            # Задержка действий до загрузки отслеживаемых елементов.
            # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
            
            if not (driver.current_url == url):
                raise MyLinkDifferError()
            
            # Парсинг
            page = BS(driver.page_source, 'html.parser')
            
            try:
                f1 = page.find('nav', {'data-test-ref':"paginator"})
            except Exception as ex:
                print(ex)
                raise ex
                continue
                
            try:    
                pagination = int(f1.find_all('a')[-2].text)
            except IndexError:
                time_sleep(0.3)
                continue
            
            out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
            
            break
        else:
            raise Exception("timeout")

        return out_list
    
    
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    for _ in range(10):
        driver.get(url)
        try:
            out_list =  data_parse(driver, wait, url, timeout)
        except MyLinkDifferError:
            time_sleep(1)
            continue
        else:
            break

    return out_list

In [48]:
loc_df = pd.read_csv('data/located_list.zip')
catalog_markets_links = list()

driver = webdriver.Chrome()
driver.maximize_window()

for index, item in loc_df.iterrows():
    url = f'https://edadeal.ru/{item.slug}/retailers'
    temp_url_list = get_pages_links(driver, url, '?page=')
    catalog_markets_links.extend(list(zip([item.slug]*len(temp_url_list), temp_url_list)))

driver.quit()
len(catalog_markets_links)

In [51]:
with open('data/catalog_markets_links.pickle', 'wb') as file:
    pickle.dump(catalog_markets_links, file)

In [52]:
with open('data/catalog_markets_links.pickle', 'rb') as file:
    catalog_markets_links = pickle.load(file)

In [66]:
def reailers_parser(page: str, host: str, sity: str):
    out_list = list()
    
    page = BS(page, 'html.parser')
        
    try:
        f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
    except AttributeError:
        raise MyNoData()
    
    try:    
        for item in f1:
            uuid = item.get('data-uuid')
            href = host + item.find('a').get('href')
            title = item.find('a').find('h4').get('title')
            discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
            out_list.append(market_link(sity, title, discounts, uuid, href))
            
    except TypeError as ex:
        raise MyNoData()

    return out_list

def get_retailers_links(driver: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1
        )

    
    # Вызов страницы
    driver.get(url)

    if (driver.find_element(By.XPATH, """//body""").text == 'Service unavailable'):
        raise MyTimeoutError('timeout')
    
    for _ in range(10):
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        except TimeoutException:
            time_sleep(0.5)
            continue
        # Парсинг
        try:
            out_list.append(reailers_parser(driver.page_source, host, sity))
        except MyNoData:
            time_sleep(0.5)
            continue
        
        break
    
    else:
        raise MyTimeoutError('timeout')
    
    return out_list

In [67]:
%%time
driver = webdriver.Chrome()
driver.maximize_window()
temp_retailer_links = list()
brake_list = list()
for sity, link in catalog_markets_links:
    for _ in range(10):
        try:
            temp_retailer_links.extend(get_retailers_links(driver, link, sity))
        except MyTimeoutError:
            continue
        else:
            break
    else:
        brake_list.append((sity, link))

driver.quit()
retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))

5159
CPU times: total: 15.8 s
Wall time: 16min 59s


In [68]:
brake_list

[]

In [69]:
retailers = pd.DataFrame(retailer_links)
retailers.insert(0, 'date', datetime.now())
retailers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5159 entries, 0 to 5158
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       5159 non-null   datetime64[ns]
 1   sity       5159 non-null   object        
 2   market     5159 non-null   object        
 3   discounts  5159 non-null   int64         
 4   data_uuid  5159 non-null   object        
 5   href       5159 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 242.0+ KB


In [70]:
_save_df_to_zip(retailers, 'retailers')
retailers.to_excel('data/retailers.xlsx', index=False)